In [ ]:
from translation import translate_fr_en_paraphrase
from transformers import pipeline
import pickle
data_path = '/data/desponds/data/Paraphrase'
%load_ext autoreload
%autoreload 2

In [ ]:
from preprocessing import preprocessing_paraphrasing
datasets, tokenized = preprocessing_paraphrasing()

In [ ]:
from training import get_trainers_paraphrasing
trainers = get_trainers_paraphrasing(tokenized,data_path,  langs = ['fr', 'en'])

In [ ]:
from training import get_models_paraphrasing
models = get_models_paraphrasing(trainers)

# Translation

In [ ]:
from translation import translate_fr_en_paraphrase
from transformers import pipeline
import pickle
data_path = '/data/desponds/data/Paraphrase'
# translator_tc_big = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-fr-en")
translator = pipeline("translation", model=f"/data/desponds/data/translation_models/model_4_4_epoch_5/checkpoint-15890")

translated_fr_en = datasets['fr']['test'].map(lambda examples : translate_fr_en_paraphrase(examples, translator), batched=True, batch_size = 512)
with open(f'{data_path}/translator_t5_small_epoch_5.pickle', 'wb') as handle:
        pickle.dump(translated_fr_en, handle)

In [ ]:
translator = pipeline("translation", model=f"/data/desponds/data/translation_models/model_t5_small_1_8_epoch_1/checkpoint-795")

translated_fr_en = datasets['fr']['test'].map(lambda examples : translate_fr_en_paraphrase(examples, translator), batched=True, batch_size = 512)
with open(f'{data_path}/translator_t5_small_1_8_epoch_1.pickle', 'wb') as handle:
        pickle.dump(translated_fr_en, handle)

## Predictions

In [ ]:
datasets, results = {}, {}
# with open(f'{data_path}/translator_t5_base_epoch_3.pickle', 'rb') as handle:
#     datasets['translated_t5_base_epoch_3'] = pickle.load(handle)
# with open(f'{data_path}/translated_Helsinki-tc-big.pickle', 'rb') as handle:
#     datasets['translated_Helsinki-tc-big'] = pickle.load(handle)
# with open(f'{data_path}/translated_dataset.pickle', 'rb') as handle:
#     datasets['translated_Helsinki'] = pickle.load(handle)
# with open(f'{data_path}/translator_t5_small_1_4_epoch_1.pickle', 'rb') as handle:
#     datasets['translator_t5_small_1_4_epoch_1'] = pickle.load(handle)
# with open(f'{data_path}/translator_t5_small_epoch_5.pickle', 'rb') as handle:
#     datasets['translator_t5_small_4_4_epoch_5'] = pickle.load(handle)
with open(f'{data_path}/translator_t5_small_1_8_epoch_1.pickle', 'rb') as handle:
    datasets['translator_t5_small_1_8_epoch_1'] = pickle.load(handle)

In [ ]:
from transformers import AutoTokenizer
from preprocessing import tokenize_paraphrasing
tokenizer = {}
tokenizer['en'] = AutoTokenizer.from_pretrained('roberta-base')

def preprocess_after_trad(dataset):
    dataset = dataset.map(lambda ex : tokenize_paraphrasing(ex, 
                                                            tokenizer['en'],
                                                           with_label = True,
                                                           truncation =True))
    return dataset

In [ ]:
for name in datasets :
    datasets[name] = preprocess_after_trad(datasets[name])
    results[name] = models['en'].predict(datasets[name])
    
results

In [ ]:
def remove_trans(example):
    example['sentence1'] = example['sentence1']['translation_text']
    example['sentence2'] = example['sentence2']['translation_text']
    return example

for name in datasets:
    datasets[name] = datasets[name].map(remove_trans)

In [ ]:
final_results = {
't5_small_0' : {'accuracy' : 0.5955,'bleu': 0.7212123764045337},
't5_small_1': {'accuracy' : 0.749, 'bleu':6.0798},
't5_small_2': {'accuracy' : 0.818, 'bleu':18.3099},
't5_small_3': {'accuracy' : 0.8235, 'bleu':20.9125},
't5_base': {'accuracy' : 0.8465, 'bleu':26.0477},
'Helsinki': {'accuracy' : 0.911, 'bleu':56.3933},
'Helsinki_big': {'accuracy' : 0.913, 'bleu':58.4968}
}
final_results

In [ ]:
import matplotlib.pyplot as plt

# Sample data
accuracies = [val['accuracy'] for key, val in final_results.items()]
bleus = [val['bleu'] for key, val in final_results.items()]
names = [key for key, val in final_results.items()]
v_shift = [0,1, -10, 0, 1, -7, -8]
v_position = ['bottom','bottom', 'top', 'bottom', 'bottom', 'top','bottom']
position = ['bottom','bottom', 'top', 'bottom', 'bottom', 'top','bottom']

# Create scatter plot
plt.scatter(bleus, accuracies)
val = 0.002
# Add text annotations for names
for i, name in enumerate(names):
    plt.text(bleus[i] +v_shift[i], accuracies[i]+val if i %2 == 0 else accuracies[i]-val, name,
             ha='left', va=position[i])

# Set plot title and labels
plt.title("Accuracies vs BLEU scores for the different translators")
plt.xlabel("BLEU score")
plt.ylabel("Accuracy")

# Display the plot
plt.show()